In [ ]:
mport torch
import torchvision
from torchvision.transforms import functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Load a pretrained Mask R-CNN model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode



In [ ]:
# Load the input image
image_path = "city.jpg"  # Change this to your image path
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR (OpenCV) to RGB



In [ ]:
# Convert image to tensor
image_tensor = F.to_tensor(image)



In [ ]:
# Perform inference
with torch.no_grad():
    predictions = model([image_tensor])  # Forward pass through the model



In [ ]:
# Extract results
boxes = predictions[0]['boxes'].numpy()  # Bounding boxes
masks = predictions[0]['masks'].detach().numpy()  # Segmentation masks
labels = predictions[0]['labels'].numpy()  # Class labels
scores = predictions[0]['scores'].numpy()  # Confidence scores



In [ ]:
# Set confidence threshold to filter out low-confidence predictions
confidence_threshold = 0.3
indices = scores > confidence_threshold

filtered_boxes = boxes[indices]
filtered_masks = masks[indices]
filtered_labels = labels[indices]


In [ ]:
# Visualizing the results
plt.figure(figsize=(10, 10))
plt.imshow(image)


In [ ]:
# Overlay bounding boxes and masks
for i in range(len(filtered_boxes)):
    box = filtered_boxes[i].astype(int)
    mask = filtered_masks[i][0]  # Extract single channel mask

    # Draw bounding box
    cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)

    # Apply mask (Threshold & colorize)
    mask = mask > 0.3
    image[mask] = (0, 255, 0)  # Color mask as green


In [ ]:
plt.imshow(image)
plt.axis("off")
plt.show()